In [1]:
import pandas as pd
import json
import numpy as np
from IPython.display import Markdown,HTML
from copy import deepcopy

In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
def load_json(path):
    with open(path, 'r') as f:
        return json.load(f)

## Load Stuff

In [4]:
raw_data = load_json('data_v2.json')

In [5]:
raw_data.keys()

dict_keys(['source', 'belts', 'pipes', 'buildings', 'miners', 'items', 'fluids', 'recipes', 'resources'])

In [6]:
key_name_fn = lambda x: x.lower().replace(' ', '-')

In [7]:
building_df = pd.DataFrame(raw_data['buildings'])

In [8]:
item_df = pd.DataFrame.from_dict(raw_data['items'])

In [9]:
fluid_df = pd.DataFrame.from_dict(raw_data['fluids'])

In [10]:
resource_df = pd.DataFrame.from_dict(raw_data['resources'])

In [11]:
recipe_df = pd.DataFrame.from_dict(raw_data['recipes'])

## Process recipe data

### Add power info

In [12]:
recipe_category_to_power = building_df.set_index('category')['power'].astype(float).to_dict()
# average of fluctuating
recipe_category_to_power.update({'converting' : 250, 'encoding' : 1000})
recipe_df['power_rate'] = recipe_df['category'].map(recipe_category_to_power)

### Recipes for Base items - e.g. ore

In [13]:
base_items = ['Iron Ore', 'Limestone', 'Copper Ore', 'Bauxite', 'Caterium Ore', 'Coal','Raw Quartz', 'Sulfur', 'Uranium'] + ['Nitrogen Gas', 'Water', 'Crude Oil', 'SAM', 'Somersloop']
base_key_items = [key_name_fn(x) for x in base_items]
base_key_items_set = set(base_key_items)

In [14]:
list(recipe_df)

['name',
 'key_name',
 'category',
 'time',
 'ingredients',
 'products',
 'power_range',
 'power_rate']

In [15]:
base_recipes = pd.DataFrame([[it, key_it, 'base', 1/1200 if not key_it in {'water', 'crude-oil', 'nitrogen-gas'} else 1/600, [], [[key_it, 1]], np.nan, 0] for (it, key_it) in zip(base_items, base_key_items)], columns = list(recipe_df))
base_recipes.shape

(14, 8)

In [16]:
base_recipes.loc[base_recipes['key_name'] == 'somersloop', 'time'] = 1
base_recipes.loc[base_recipes['key_name'] == 'power-shard', 'time'] = 1

In [17]:
base_recipes['power_rate'] = [45, 45, 45, 45, 45, 45, 45, 45, 45, 150, 20, 40, 45, 0]

In [18]:
recipe_df = pd.concat([recipe_df, base_recipes])

In [19]:
base_recipes

,name,key_name,category,time,ingredients,products,power_range,power_rate
0,Iron Ore,iron-ore,base,0.000833,[],"[[iron-ore, 1]]",NaN,45
1,Limestone,limestone,base,0.000833,[],"[[limestone, 1]]",NaN,45
2,Copper Ore,copper-ore,base,0.000833,[],"[[copper-ore, 1]]",NaN,45
3,Bauxite,bauxite,base,0.000833,[],"[[bauxite, 1]]",NaN,45
4,Caterium Ore,caterium-ore,base,0.000833,[],"[[caterium-ore, 1]]",NaN,45
5,Coal,coal,base,0.000833,[],"[[coal, 1]]",NaN,45
6,Raw Quartz,raw-quartz,base,0.000833,[],"[[raw-quartz, 1]]",NaN,45
7,Sulfur,sulfur,base,0.000833,[],"[[sulfur, 1]]",NaN,45
8,Uranium,uranium,base,0.000833,[],"[[uranium, 1]]",NaN,45
9,Nitrogen Gas,nitrogen-gas,base,0.001667,[],"[[nitrogen-gas, 1]]",NaN,150


### Power Reecipes

In [20]:
# items per minute
power_recipes = pd.DataFrame([
                                ['coal-coal-power', [['coal', 15], ['water', 45]], [], -75],
                                ['coal-compacted-coal-power', [['compacted-coal', 7 + 1/7], ['water', 45]], [], -75],
                                ['coal-petroleum-coke-power', [['petroleum-coke', 25], ['water', 45]], [], -75],
    
                                ['fuel-powered-fuel-power', [['fuel', 20]], [], -250],
                                ['fuel-powered-liquid-biofuel-power', [['liquid-biofuel', 20]],[], -250],
                                ['fuel-powered-turbofuel-power', [['turbofuel', 7.5]], [], -250],
                                ['fuel-powered-rocket-fuel-power', [['rocket-fuel', 4 + 1/6]], [], -250],
                                ['fuel-powered-ionized-fuel-power', [['ionized-fuel', 3]], [], -250],
                                ['nuclear-uranium-power', [['water', 240], ['uranium-fuel-rod', .3]], [['uranium-waste', 10]], -2500],
                                ['nuclear-plutonium-power', [['water', 240], ['plutonium-fuel-rod', .1]], [['plutonium-waste', 1]], -2500],
                                ['nuclear-ficsonium-power', [['water', 240], ['ficsonium-fuel-rod', 1]], [], -2500],
                             ],columns = ['key_name', 'ingredients', 'products', 'power_rate'])
power_recipes['name'] = power_recipes['key_name']
power_recipes['category'] = 'power'
power_recipes['power_range'] = np.nan
power_recipes['time'] = 60

In [21]:
power_recipes

,key_name,ingredients,products,power_rate,name,category,power_range,time
0,coal-coal-power,"[[coal, 15], [water, 45]]",[],-75,coal-coal-power,power,NaN,60
1,coal-compacted-coal-power,"[[compacted-coal, 7.142857142857143], [water, ...",[],-75,coal-compacted-coal-power,power,NaN,60
2,coal-petroleum-coke-power,"[[petroleum-coke, 25], [water, 45]]",[],-75,coal-petroleum-coke-power,power,NaN,60
3,fuel-powered-fuel-power,"[[fuel, 20]]",[],-250,fuel-powered-fuel-power,power,NaN,60
4,fuel-powered-liquid-biofuel-power,"[[liquid-biofuel, 20]]",[],-250,fuel-powered-liquid-biofuel-power,power,NaN,60
5,fuel-powered-turbofuel-power,"[[turbofuel, 7.5]]",[],-250,fuel-powered-turbofuel-power,power,NaN,60
6,fuel-powered-rocket-fuel-power,"[[rocket-fuel, 4.166666666666667]]",[],-250,fuel-powered-rocket-fuel-power,power,NaN,60
7,fuel-powered-ionized-fuel-power,"[[ionized-fuel, 3]]",[],-250,fuel-powered-ionized-fuel-power,power,NaN,60
8,nuclear-uranium-power,"[[water, 240], [uranium-fuel-rod, 0.3]]","[[uranium-waste, 10]]",-2500,nuclear-uranium-power,power,NaN,60
9,nuclear-plutonium-power,"[[water, 240], [plutonium-fuel-rod, 0.1]]","[[plutonium-waste, 1]]",-2500,nuclear-plutonium-power,power,NaN,60


In [22]:
recipe_df.shape, power_recipes.shape

((286, 8), (11, 8))

In [23]:
recipe_df = pd.concat([recipe_df, power_recipes], axis=0)

In [24]:
recipe_df

,name,key_name,category,time,ingredients,products,power_range,power_rate
0,Iron Ingot,iron-ingot,smelting1,2.0,"[[iron-ore, 1]]","[[iron-ingot, 1]]",NaN,4.0
1,Alternate: Iron Alloy Ingot,alt-iron-ingot,smelting2,12.0,"[[iron-ore, 8], [copper-ore, 2]]","[[iron-ingot, 15]]",NaN,16.0
2,Alternate: Pure Iron Ingot,alt-pure-iron-ingot,refining,12.0,"[[iron-ore, 7], [water, 4]]","[[iron-ingot, 13]]",NaN,30.0
3,Alternate: Leached Iron Ingot,alt-leached-iron-ingot,refining,6.0,"[[iron-ore, 5], [sulfuric-acid, 1]]","[[iron-ingot, 10]]",NaN,30.0
4,Copper Ingot,copper-ingot,smelting1,2.0,"[[copper-ore, 1]]","[[copper-ingot, 1]]",NaN,4.0
...,...,...,...,...,...,...,...,...
6,fuel-powered-rocket-fuel-power,fuel-powered-rocket-fuel-power,power,60.0,"[[rocket-fuel, 4.166666666666667]]",[],NaN,-250.0
7,fuel-powered-ionized-fuel-power,fuel-powered-ionized-fuel-power,power,60.0,"[[ionized-fuel, 3]]",[],NaN,-250.0
8,nuclear-uranium-power,nuclear-uranium-power,power,60.0,"[[water, 240], [uranium-fuel-rod, 0.3]]","[[uranium-waste, 10]]",NaN,-2500.0
9,nuclear-plutonium-power,nuclear-plutonium-power,power,60.0,"[[water, 240], [plutonium-fuel-rod, 0.1]]","[[plutonium-waste, 1]]",NaN,-2500.0


### Add Recipes for poower slugs/sommerslops'\

In [25]:
slooped_recipes = recipe_df.copy()

In [26]:
nonsloopable = ['base', 'power', 'packaging', 'nuke-reacting']
sloop_costs = {
    'crafting1' : 1,
    'crafting2' : 2,
    'crafting3' : 4,
    'refining' : 2,
    'converting' : 2,
    'blending' : 4,
    'accelerating' : 4,
    'smelting1' : 1,
    'smelting2' : 2,
    'encoding' : 4,
}

In [27]:
slooped_recipes['category'].value_counts(dropna=False)

category
crafting2        60
crafting1        43
crafting3        37
refining         34
converting       25
packaging        24
blending         16
base             14
accelerating     12
power            11
smelting2         9
encoding          6
smelting1         4
nuke-reacting     2
Name: count, dtype: int64

In [28]:
slooped_recipes = slooped_recipes[~slooped_recipes['category'].isin(nonsloopable)]

In [29]:
slooped_recipes[slooped_recipes['category'] == 'crafting2']

,name,key_name,category,time,ingredients,products,power_range,power_rate
14,Alternate: Coated Iron Plate,alt-iron-plate,crafting2,8.0,"[[iron-ingot, 5], [plastic, 1]]","[[iron-plate, 10]]",NaN,15.0
21,Alternate: Fused Wire,alt-fused-wire,crafting2,20.0,"[[copper-ingot, 4], [caterium-ingot, 1]]","[[wire, 30]]",NaN,15.0
23,Alternate: Insulated Cable,alt-cable,crafting2,12.0,"[[wire, 9], [rubber, 6]]","[[cable, 20]]",NaN,15.0
24,Alternate: Quickwire Cable,alt-quickwire-cable,crafting2,24.0,"[[quickwire, 3], [rubber, 2]]","[[cable, 11]]",NaN,15.0
27,Alternate: Fine Concrete,alt-concrete,crafting2,12.0,"[[silica, 3], [limestone, 12]]","[[concrete, 10]]",NaN,15.0
28,Alternate: Rubber Concrete,alt-rubber-concrete,crafting2,6.0,"[[limestone, 10], [rubber, 2]]","[[concrete, 9]]",NaN,15.0
51,Reinforced Iron Plate,reinforced-iron-plate,crafting2,12.0,"[[iron-plate, 6], [screw, 12]]","[[reinforced-iron-plate, 1]]",NaN,15.0
52,Alternate: Bolted Iron Plate,alt-bolted-iron-plate,crafting2,12.0,"[[iron-plate, 18], [screw, 50]]","[[reinforced-iron-plate, 3]]",NaN,15.0
53,Alternate: Stitched Iron Plate,alt-stitched-iron-plate,crafting2,32.0,"[[iron-plate, 10], [wire, 20]]","[[reinforced-iron-plate, 3]]",NaN,15.0
54,Alternate: Adhered Iron Plate,alt-adhered-iron-plate,crafting2,16.0,"[[iron-plate, 3], [rubber, 1]]","[[reinforced-iron-plate, 1]]",NaN,15.0


In [30]:
slooped_recipes['key_name'] = slooped_recipes['key_name'].apply(lambda x: x + '-slooped')

In [31]:
slooped_recipes['ingredients'] = slooped_recipes.apply(lambda x: x['ingredients'] + [['somersloop', sloop_costs[x['category']]]], axis=1)
slooped_recipes['products'] = slooped_recipes['products'].apply(lambda x: [[y[0], y[1] * 2] for y in x])
slooped_recipes['power_rate'] = slooped_recipes['power_rate'] * 4

In [32]:
slooped_recipes

,name,key_name,category,time,ingredients,products,power_range,power_rate
0,Iron Ingot,iron-ingot-slooped,smelting1,2.0,"[[iron-ore, 1], [somersloop, 1]]","[[iron-ingot, 2]]",NaN,16.0
1,Alternate: Iron Alloy Ingot,alt-iron-ingot-slooped,smelting2,12.0,"[[iron-ore, 8], [copper-ore, 2], [somersloop, 2]]","[[iron-ingot, 30]]",NaN,64.0
2,Alternate: Pure Iron Ingot,alt-pure-iron-ingot-slooped,refining,12.0,"[[iron-ore, 7], [water, 4], [somersloop, 2]]","[[iron-ingot, 26]]",NaN,120.0
3,Alternate: Leached Iron Ingot,alt-leached-iron-ingot-slooped,refining,6.0,"[[iron-ore, 5], [sulfuric-acid, 1], [somersloo...","[[iron-ingot, 20]]",NaN,120.0
4,Copper Ingot,copper-ingot-slooped,smelting1,2.0,"[[copper-ore, 1], [somersloop, 1]]","[[copper-ingot, 2]]",NaN,16.0
...,...,...,...,...,...,...,...,...
265,Bauxite (Copper),bauxite-copper-slooped,converting,6.0,"[[reanimated-sam, 1], [copper-ore, 18], [somer...","[[bauxite, 24]]",NaN,1000.0
266,Bauxite (Caterium),bauxite-caterium-slooped,converting,6.0,"[[reanimated-sam, 1], [caterium-ore, 15], [som...","[[bauxite, 24]]",NaN,1000.0
267,Nitrogen Gas (Caterium),nitrogen-gas-caterium-slooped,converting,6.0,"[[reanimated-sam, 1], [caterium-ore, 12], [som...","[[nitrogen-gas, 24]]",NaN,1000.0
268,Nitrogen Gas (Bauxite),nitrogen-gas-bauxite-slooped,converting,6.0,"[[reanimated-sam, 1], [bauxite, 10], [somerslo...","[[nitrogen-gas, 24]]",NaN,1000.0


In [33]:
recipe_df = pd.concat([recipe_df, slooped_recipes])

In [34]:
recipe_df

,name,key_name,category,time,ingredients,products,power_range,power_rate
0,Iron Ingot,iron-ingot,smelting1,2.0,"[[iron-ore, 1]]","[[iron-ingot, 1]]",NaN,4.0
1,Alternate: Iron Alloy Ingot,alt-iron-ingot,smelting2,12.0,"[[iron-ore, 8], [copper-ore, 2]]","[[iron-ingot, 15]]",NaN,16.0
2,Alternate: Pure Iron Ingot,alt-pure-iron-ingot,refining,12.0,"[[iron-ore, 7], [water, 4]]","[[iron-ingot, 13]]",NaN,30.0
3,Alternate: Leached Iron Ingot,alt-leached-iron-ingot,refining,6.0,"[[iron-ore, 5], [sulfuric-acid, 1]]","[[iron-ingot, 10]]",NaN,30.0
4,Copper Ingot,copper-ingot,smelting1,2.0,"[[copper-ore, 1]]","[[copper-ingot, 1]]",NaN,4.0
...,...,...,...,...,...,...,...,...
265,Bauxite (Copper),bauxite-copper-slooped,converting,6.0,"[[reanimated-sam, 1], [copper-ore, 18], [somer...","[[bauxite, 24]]",NaN,1000.0
266,Bauxite (Caterium),bauxite-caterium-slooped,converting,6.0,"[[reanimated-sam, 1], [caterium-ore, 15], [som...","[[bauxite, 24]]",NaN,1000.0
267,Nitrogen Gas (Caterium),nitrogen-gas-caterium-slooped,converting,6.0,"[[reanimated-sam, 1], [caterium-ore, 12], [som...","[[nitrogen-gas, 24]]",NaN,1000.0
268,Nitrogen Gas (Bauxite),nitrogen-gas-bauxite-slooped,converting,6.0,"[[reanimated-sam, 1], [bauxite, 10], [somerslo...","[[nitrogen-gas, 24]]",NaN,1000.0


In [35]:
recipe_df[recipe_df['key_name'] == 'iron-ingot-slooped']

,name,key_name,category,time,ingredients,products,power_range,power_rate
0,Iron Ingot,iron-ingot-slooped,smelting1,2.0,"[[iron-ore, 1], [somersloop, 1]]","[[iron-ingot, 2]]",NaN,16.0


In [36]:
overclocked = recipe_df.copy()

In [37]:
overclocked = overclocked[~(overclocked['key_name'].apply(lambda x: 'power-shard' in x or 'somersloop' in x))]

In [38]:
overclocked['key_name'] = overclocked['key_name'].apply(lambda x: x + '-overclocked')
overclocked['ingredients'] = overclocked['ingredients'].apply(lambda x: [[y[0], (y[1] * (2.5 if not y[0] in {'somersloop'} else 1))] for y in x])
# overclocked['ingredients'] = overclocked['ingredients'].apply(lambda x: x + [['power-shard', 3]])
overclocked['products'] = overclocked['products'].apply(lambda x: [[y[0], (y[1] * (2.5 if not y[1] in {'somersloop'} else 1))] for y in x])
overclocked.loc[overclocked['category'] == 'power', 'power_rate'] *= 2.5
overclocked.loc[~(overclocked['category'] == 'power'), 'power_rate'] *= 2.5**1.321928

In [39]:
overclocked[overclocked['key_name'] == 'iron-ingot-slooped-overclocked']

,name,key_name,category,time,ingredients,products,power_range,power_rate
0,Iron Ingot,iron-ingot-slooped-overclocked,smelting1,2.0,"[[iron-ore, 2.5], [somersloop, 1]]","[[iron-ingot, 5.0]]",NaN,53.723961


In [40]:
recipe_df = pd.concat([recipe_df, overclocked])

In [41]:
sys.exit()

NameError: name 'sys' is not defined

In [42]:
# overclock = 'base'
# sloop = 'base'

# if not overclock in {'max', 'base'} and 'sloop' in {'max', 'base'}:
#     raise ValueError

In [43]:
# 2.5**1.321928

In [44]:
# recipe_df['category'].value_counts()

In [45]:
# # 2.5**1.321928 from: https://satisfactory.wiki.gg/wiki/Clock_speed

# if overclock == 'max':
#     recipe_df['ingredients'] = recipe_df['ingredients'].apply(lambda x: [[y[0], y[1] * 2.5] for y in x])
#     recipe_df['products'] = recipe_df['products'].apply(lambda x: [[y[0], y[1] * 2.5] for y in x])
#     recipe_df['power_rate'] = recipe_df['power_rate'] * 2.5**1.321928
# if sloop == 'max':
#     non_base_idxr = recipe_df['category'] != 'base'
#     recipe_df.loc[non_base_idxr,'products'] = recipe_df.loc[non_base_idxr]['products'].apply(lambda x: [[y[0], y[1] * 2] for y in x])
#     recipe_df.loc[non_base_idxr,'power_rate'] = recipe_df.loc[non_base_idxr]['power_rate'] * 4
    

### load/process points data

In [46]:
points_df = pd.read_csv('points.csv')

In [47]:
points_df

,Points,Items
0,Cannot be sunk,"Alien Protein, Bacon Agaric, Beryl Nut, Bl..."
1,1,"Blue FICSMAS Ornament, FICSIT Coupon, FICSM..."
2,2,"FICSMAS Tree Branch, Iron Ingot, Limestone,..."
3,3,"Coal, Copper Ore, Leaves"
4,4,"FICSMAS Bow, Iron Rod"
...,...,...
120,"500,176",Assembly Director System
121,"538,976",Nuclear Pasta
122,"597,652",AI Expansion Server
123,"728,508",Thermal Propulsion Rocket


In [48]:
list(points_df)

['Points ', 'Items']

In [49]:
# split stuff as list by comma & explode
points_df['Items'] = points_df['Items'].apply(lambda x: x.split(','))
points_df = points_df.explode('Items').reset_index(drop=True)
points_df.rename({'Points ' : 'points', 'Items' : 'item'}, axis=1, inplace=True)
points_df = points_df[points_df['points'] != 'Cannot be sunk ']
points_df['item'] = points_df['item'].apply(lambda x: x.strip())
points_df['points'] = points_df['points'].apply(lambda x: int(x.replace(',', '')))

In [50]:
points_df

,points,item
22,1,Blue FICSMAS Ornament
23,1,FICSIT Coupon
24,1,FICSMAS Gift
25,1,Iron Ore
26,2,FICSMAS Tree Branch
...,...,...
174,500176,Assembly Director System
175,538976,Nuclear Pasta
176,597652,AI Expansion Server
177,728508,Thermal Propulsion Rocket


In [51]:
# points_df[points_df['item'] == 'SAM Fluctuator']

In [52]:
garbage = ["Actual Snow",
"Candy Cane",
"FICSMAS Gift",
"Sweet Fireworks",
"Sparkly Fireworks",
"Copper FICSMAS Ornament",
"FICSMAS Ornament Bundle",
"Iron FICSMAS Ornament",
"Snowball",
"Red FICSMAS Ornament",
"FICSMAS Wonder Star",
"Fancy Fireworks",
"FICSMAS Bow",
'Portable Miner']

In [53]:
points_df = points_df[points_df['item'].apply(lambda x: not x in garbage)].reset_index(drop=True)

In [54]:
points_df['key_name'] = points_df['item'].apply(key_name_fn)

In [55]:
recipe_df

,name,key_name,category,time,ingredients,products,power_range,power_rate
0,Iron Ingot,iron-ingot,smelting1,2.0,"[[iron-ore, 1]]","[[iron-ingot, 1]]",NaN,4.000000
1,Alternate: Iron Alloy Ingot,alt-iron-ingot,smelting2,12.0,"[[iron-ore, 8], [copper-ore, 2]]","[[iron-ingot, 15]]",NaN,16.000000
2,Alternate: Pure Iron Ingot,alt-pure-iron-ingot,refining,12.0,"[[iron-ore, 7], [water, 4]]","[[iron-ingot, 13]]",NaN,30.000000
3,Alternate: Leached Iron Ingot,alt-leached-iron-ingot,refining,6.0,"[[iron-ore, 5], [sulfuric-acid, 1]]","[[iron-ingot, 10]]",NaN,30.000000
4,Copper Ingot,copper-ingot,smelting1,2.0,"[[copper-ore, 1]]","[[copper-ingot, 1]]",NaN,4.000000
...,...,...,...,...,...,...,...,...
265,Bauxite (Copper),bauxite-copper-slooped-overclocked,converting,6.0,"[[reanimated-sam, 2.5], [copper-ore, 45.0], [s...","[[bauxite, 60.0]]",NaN,3357.747541
266,Bauxite (Caterium),bauxite-caterium-slooped-overclocked,converting,6.0,"[[reanimated-sam, 2.5], [caterium-ore, 37.5], ...","[[bauxite, 60.0]]",NaN,3357.747541
267,Nitrogen Gas (Caterium),nitrogen-gas-caterium-slooped-overclocked,converting,6.0,"[[reanimated-sam, 2.5], [caterium-ore, 30.0], ...","[[nitrogen-gas, 60.0]]",NaN,3357.747541
268,Nitrogen Gas (Bauxite),nitrogen-gas-bauxite-slooped-overclocked,converting,6.0,"[[reanimated-sam, 2.5], [bauxite, 25.0], [some...","[[nitrogen-gas, 60.0]]",NaN,3357.747541


In [56]:
points_df

,points,item,key_name
0,1,Blue FICSMAS Ornament,blue-ficsmas-ornament
1,1,FICSIT Coupon,ficsit-coupon
2,1,Iron Ore,iron-ore
3,2,FICSMAS Tree Branch,ficsmas-tree-branch
4,2,Iron Ingot,iron-ingot
...,...,...,...
138,500176,Assembly Director System,assembly-director-system
139,538976,Nuclear Pasta,nuclear-pasta
140,597652,AI Expansion Server,ai-expansion-server
141,728508,Thermal Propulsion Rocket,thermal-propulsion-rocket


### Filter out unwanted things form poitns & recipes

In [57]:
ignore_things = [key_name_fn(x) for x in ['Hog Remains', 'Spitter Remains', 'Stinger Remains', 'Hatcher Remains', 'Leaves', 'Wood', 'Mycelia', 'Yellow Power Slug', 'Purple Power Slug', 'Blue Power Slug', 'Automated Miner', 'Portable Miner', 'Alien DNA Capsule', 'Synthetic Power Shard']]
ignore_things = ignore_things + [x + '-slooped' for x in ignore_things]

In [58]:
print(recipe_df.shape)
recipe_df = recipe_df[recipe_df['products'].apply(lambda x: not any([y[0] in ignore_things for y in x]))].reset_index(drop=True)
recipe_df = recipe_df[recipe_df['ingredients'].apply(lambda x: not any([y[0] in ignore_things for y in x]))].reset_index(drop=True)
recipe_df = recipe_df[~recipe_df['key_name'].isin(ignore_things)].reset_index(drop=True)
recipe_df.shape

(1077, 8)


(1025, 8)

In [59]:
print(points_df.shape)
points_df = points_df[~points_df['key_name'].isin(ignore_things)].reset_index(drop=True)
points_df.shape

(143, 3)


(140, 3)

### Make points data & recipe data joinable

In [60]:
points_rename_mapper = {
    'encased-uranium-cell' : 'uranium-cell',
#     'biomass-(leaves)' : 'biomass-from-leaves',
#     'biomass-(mycelia)' : 'biomass-from-mycelia',
#     'biomass-(wood)' : 'biomass-from-wood',
    'gas-filter' : 'filter',
    'electromagnetic-control-rod' : 'em-control-rod',
    
}
points_df['key_name'] = points_df['key_name'].apply(lambda x: points_rename_mapper.get(x, x))

### Validate that everything is good
We need:  
* all ingredients/products that should be scoreable are scoreable
* WHAT ELSE DO WE NEED???

In [61]:
NO_SINK_ENTRY = set([key_name_fn(x) for x in ['Alien Protein', 'Encased Plutonium Cell', 'Ficsonium', 'Ficsonium Fuel Rod', 'Non Fissile Uranium', 'Plutonium Pellet', 'Plutonium Waste', 'Power Shard', 'Uranium Waste', 'Somersloop']] + fluid_df['key_name'].tolist())

In [62]:
all_ingredients = set(recipe_df['ingredients'].explode().dropna().apply(lambda x: x[0] if len(x) else None).dropna().unique())
all_products = set(recipe_df['products'].explode().dropna().apply(lambda x: x[0] if len(x) else None).dropna().unique())

In [63]:
all_scoreables = set(points_df['key_name'].tolist())

In [64]:
len(all_ingredients), len(all_products), len(all_scoreables)

(126, 138, 140)

In [65]:
missing_scores = (all_ingredients | all_products) - all_scoreables - NO_SINK_ENTRY

In [66]:
if len(missing_scores):
    print('MISSING: ', missing_scores)
    raise ValueError

In [67]:
recipe_df[recipe_df['key_name'].apply(lambda x: 'power-shard' in x)]

,name,key_name,category,time,ingredients,products,power_range,power_rate


### Setup/Solve LP!

In [68]:
"""
how on earth do I make a linear program again???

scipy API:
    params: A_ub, b_ub, A_eq, b_eq, (lb, ub)
    integrality: [0,3]; each int is method of considering problem; 3:= integers & 0 acceptable

    min c.T @ X
    s.t.
    A_{ub}x <= b_{ub}
    A_{eq}x == b_{eq}
    lb <= x <= ub


X: baseline: how many of each item am I making
C: associated score

Issue:
    items get used as ingredients don't get scored, but you'd want to use them potentially
    So need to duplicate all items:
        [items1, items2]
        items

"""


"\nhow on earth do I make a linear program again???\n\nscipy API:\n    params: A_ub, b_ub, A_eq, b_eq, (lb, ub)\n    integrality: [0,3]; each int is method of considering problem; 3:= integers & 0 acceptable\n\n    min c.T @ X\n    s.t.\n    A_{ub}x <= b_{ub}\n    A_{eq}x == b_{eq}\n    lb <= x <= ub\n\n\nX: baseline: how many of each item am I making\nC: associated score\n\nIssue:\n    items get used as ingredients don't get scored, but you'd want to use them potentially\n    So need to duplicate all items:\n        [items1, items2]\n        items\n\n"

In [69]:
base_inputs_test = {
    'iron-ore' : 1000,
    'coal' : 600,
    'water' : 1000,
    'somersloop' : 100,
#     'power-shard' : 100,
}
base_inputs_irl = {
    'coal' : 42300,
    'crude-oil' : 12600,
    'nitrogen-gas' : 12000,
    'bauxite' : 12300,
    'copper-ore': 36900,
    'caterium-ore' : 15000,
    'iron-ore' : 92100,
    'uranium' : 2100,
    'raw-quartz' : 13500,
    'sam' : 10200,
    'limestone' : 69300,
    'sulfur' : 10800,
    'water' : 1312500,
    'excited-photonic-matter' : 10000,
    'somersloop' : 106,
#     'power-shard' : 20_000_000
}

for key in base_items:
    key = key.lower().replace(' ', '-')
    if key in base_inputs_irl:
        continue
    base_inputs_irl[key] = 0


base_inputs_irl = base_inputs_irl | {x_processed : 0 for x in base_items if (not (x_processed:=x.lower().replace(' ', '-'))) in base_inputs_irl}

In [70]:
from scipy.optimize import linprog

In [71]:
import math

In [72]:
#gpt4o couldn't write this function, but claude could
def style_column(col, logspace=False):
    """
    Style a column with color gradient from red (min) to green (max)
    Args:
        col: pandas Series with numeric values
    Returns:
        List of CSS background-color styles
    """
    # Normalize values between 0 and 1
    if pd.isnull(col).all():
        return [''] * col.shape[0]
    if col.shape[0] == 1:
        r = 0
        g = 255
        return [f'background-color:rgba({r}, {g}, 0, 0.5)']
    if logspace:
        col = col.apply(lambda x: np.log(1 + x))
    min_val = col.min()
    max_val = col.max()
    norm = (col - min_val) / (max_val - min_val)
    
    # Convert to RGB colors (red to green)
    def get_color(value):
        if pd.isnull(value):
            return ''
        r = int(255 * (1 - value))
        g = int(255 * value)
        return f'background-color:rgba({r}, {g}, 0, 0.5)'
    
    return [get_color(x) for x in norm]

In [73]:
def lp_solve(pdf, rdf, base_inputs, force_excess_unsinkable_usage = 'all', sort_by_col='all'):
    valid_sort_by_col= ['qty_produce', 'score_output', 'power_work']
    if not sort_by_col in valid_sort_by_col + ['all']:
        raise ValueError(f'sort by col must be in {valid_sort_by_col} or "all"')
    valid_excess_unsinkable = {'all', 'waste', None, ''}
    if not force_excess_unsinkable_usage in valid_excess_unsinkable:
        raise ValueError(f'force_excess_unsinkable_usage must be in {valid_excess_unsinkable} or "all"')
    
    pdf = pdf.copy()
    point_mapper = pdf.set_index('key_name')['points'].to_dict()
    
    rdf = rdf.copy()
    if not rdf['key_name'].nunique() == rdf.shape[0]:
        raise ValueError('"key_name" in rrecipe df assumed to be unique, but its not!!!')
        
    all_recipe_items = sorted(list(set(rdf['ingredients'].explode().dropna().apply(lambda x: x[0]).tolist() + rdf['products'].explode().dropna().apply(lambda x: x[0]).tolist())))
    all_recipe_items_set = set(all_recipe_items)
        
    base_inputs = {k : base_inputs[k] for k in sorted(base_inputs) if k in all_recipe_items_set}
#     print(base_inputs)
#     sys.exit()
    
    #pdf = points_df, rdf=recipe_df
    
    
    
    
    
    num_items_with_points = np.sum([item in point_mapper for item in all_recipe_items])
    trash_df = pd.DataFrame.from_records([[f'{item} Trash', f'{item}-trash', 'sink', 0, [[item, 1]], [], np.nan, 0] for item in all_recipe_items if item in point_mapper], columns = list(rdf))
    rdf = pd.concat([rdf, trash_df], axis=0).reset_index(drop=True)    
    rdf['id'] = np.arange(rdf.shape[0])
    
    recipe_to_col_idx = {name : i for i, name in enumerate(rdf['key_name'].tolist())}

#     display(Markdown('## Inputs'))
#     display(pdf)
#     display(rdf)
#     sys.exit()
    
    
    recipe_constraint_row_order = {ing : i for i, ing in enumerate(all_recipe_items)}
    inv_recipe_constraint_row_order = {ing : i for i, ing in enumerate(all_recipe_items)}
    
    """constraints for recipes; +  1 for power"""
    constraint_matrix = np.zeros((len(all_recipe_items) + 1, rdf.shape[0]))
    constraint_values = np.zeros(len(all_recipe_items) + 1)
    def build_recipe_constraints(row):
        ingr_list = row['ingredients']
        products = row['products']
        col_idx = row['id']
        
        if len(ingr_list):
            for ingr, qty in ingr_list:
                constraint_matrix[recipe_constraint_row_order[ingr], col_idx] = qty
        if len(products):
            for prod, qty in products:
                constraint_matrix[recipe_constraint_row_order[prod], col_idx] = -qty
        
        constraint_matrix[-1, col_idx] = row['power_rate'] * row['time']
        
    rdf.apply(build_recipe_constraints, axis=1)
    
    
    """constraints for raw materials"""
    raw_material_constraints = np.zeros((len(base_inputs), rdf.shape[0]))
    raw_material_constraint_values = np.zeros(len(base_inputs))
    
    for i, (item, amount) in enumerate(base_inputs.items()):
        raw_material_constraints[i, recipe_to_col_idx[item]] = 1
        raw_material_constraint_values[i] = amount
        
        overclocked_item = item + '-overclocked'
        if overclocked_item in recipe_to_col_idx:
            raw_material_constraints[i, recipe_to_col_idx[overclocked_item]] = 2.5
            
    
    



#     print(constraint_matrix.shape, constraint_values.shape)
    constraint_matrix = np.concatenate((constraint_matrix, raw_material_constraints), axis=0)
    constraint_values = np.concatenate((constraint_values, raw_material_constraint_values), axis=0)
#     print(constraint_matrix.shape, constraint_values.shape)
#     sys.exit()
    
    
            
            
#     print(constraint_matrix)
#     print(constraint_values)

    maximize_vec = np.array([0] *(rdf.shape[0] - num_items_with_points) + [point_mapper[item] for item in all_recipe_items if item in point_mapper])

    constraint_df = pd.DataFrame(np.concatenate((np.concatenate((constraint_matrix, constraint_values[:,np.newaxis]), axis=1), np.concatenate((maximize_vec, [np.nan]))[np.newaxis,:]), axis=0), index = all_recipe_items + ['POWER'] + list(base_inputs.keys()) + ['**POINTS**'], columns = rdf['key_name'].tolist() + ['CONSTRAINT_VALUE'])#.reset_index()
#     constraint_df.rename({'index' : 'resource'}, axis=1)
    
    display(Markdown('## Constraints'))
#     styled = constraint_df.style.apply(highlight_last_row_and_col, axis=None)
    with pd.option_context('display.max_rows', 100, 'display.max_columns', 100):
        display(constraint_df)
    
#     sys.exit()
#     display(constraint_df.style.apply(style_column))
    
    lp_options = {'disp' : True, 'time_limit':300}
    
    if force_excess_unsinkable_usage:
        constraint_is_waste = np.zeros(constraint_matrix.shape[0], dtype=bool)
        debugging_matches = []
        
        for ing, idx in recipe_constraint_row_order.items():
            if force_excess_unsinkable_usage == 'all':
                if not ing in point_mapper:
                    constraint_is_waste[idx] = True
                    debugging_matches.append(ing)
            elif force_excess_unsinkable_usage == 'waste':
                if 'waste' in ing:
                    constraint_is_waste[idx] = True
                    debugging_matches.append(ing)
                
        
        if constraint_is_waste.sum():
            print('FOUND WASTE: ', debugging_matches)
#             sys.exit()
            lp_res = linprog(-maximize_vec, A_ub=constraint_matrix[~constraint_is_waste], b_ub=constraint_values[~constraint_is_waste], A_eq=constraint_matrix[constraint_is_waste], b_eq=constraint_values[constraint_is_waste], bounds=(0, None), method='highs', callback=None, options=lp_options, x0=None, integrality=3)
            
        else:
#             print('NO WASTE FOUND!!!')
#             sys.exit()
            lp_res = linprog(-maximize_vec, A_ub=constraint_matrix, b_ub=constraint_values, A_eq=None, b_eq=None, bounds=(0, None), method='highs', callback=None, options=lp_options, x0=None, integrality=3)
        
        
    else:
        lp_res = linprog(-maximize_vec, A_ub=constraint_matrix, b_ub=constraint_values, A_eq=None, b_eq=None, bounds=(0, None), method='highs', callback=None, options=lp_options, x0=None, integrality=3)
    
    values = lp_res.x
    
    rdf['qty_produce'] = values
    if rdf['qty_produce'].isnull().all():
        display(Markdown('## FAIL'))
#         display(rdf)
        print(lp_res.message)
        return lp_res, rdf
    
    # "[:-6]" --> removes "-trash" suffix
#     display(rdf)
    rdf['score_output'] = rdf.apply(lambda x: 0 if not x['key_name'].endswith('-trash') else x['qty_produce'] * point_mapper[x['key_name'][:-6]], axis=1)
    rdf['power_work'] = rdf['qty_produce'] * rdf['power_rate']
    
#     display(rdf)
    
    display(Markdown('## Output'))
    if sort_by_col == 'all':
        for c in valid_sort_by_col:
            display(Markdown(f'#### {c}'))
            display(rdf[rdf[c] > 0].sort_values(c, ascending=False).style.apply(style_column, subset=[c]))
    else:
        display(rdf[rdf[sort_by_col] > 0].sort_values(sort_by_col, ascending=False).style.apply(style_column, subset=[sort_by_col]))
        
    display(Markdown(f'#### slooped'))
    display(rdf[rdf.apply(lambda x: '-slooped' in x['key_name'] and x['qty_produce'] > 0, axis=1)].sort_values('qty_produce').style.apply(style_column, subset='qty_produce'))
    
    return lp_res, rdf
    

In [74]:
# recipe_df[recipe_df['key_name'].apply(lambda x: 'coal' in x)]

In [75]:
# recipe_df[recipe_df['category'] == 'power']

In [76]:
eg_stuff = {'iron-ore', 'iron-ingot', 'iron-plate', 'iron-rod', 'coal', 'coal-coal-power', 'water', 'somersloop', 'power-shard'}
eg_stuff = eg_stuff | {x + '-slooped' for x in eg_stuff}
eg_stuff = eg_stuff | {x + '-overclocked' for x in eg_stuff}

eg_points = points_df[points_df['key_name'].isin(eg_stuff)].reset_index(drop=True).copy()
eg_recipes = recipe_df[recipe_df['key_name'].isin(eg_stuff)].reset_index(drop=True).copy()

# eg_points = points_df[points_df['key_name'].isin({'iron-ore', 'iron-ingot', 'iron-plate'})].reset_index(drop=True).copy()
# eg_recipes = recipe_df[recipe_df['key_name'].isin({'iron-ingot', 'iron-plate', 'iron-ore'})].reset_index(drop=True).copy()

# eg_points = points_df[points_df['key_name'].isin({'iron-ore', 'iron-ingot'})].reset_index(drop=True).copy()
# eg_recipes = recipe_df[recipe_df['key_name'].isin({'iron-ingot', 'iron-ore'})].reset_index(drop=True).copy()

In [77]:
# eg_recipes[eg_recipes['key_name'].apply(lambda x: 'sloop' in x)]

In [82]:
# eg_recipes

In [83]:
# sys.exit()

In [84]:
# res = lp_solve(eg_points, eg_recipes, base_inputs_test, force_excess_unsinkable_usage='all', sort_by_col = 'all')

In [ ]:
sys.exit()

In [81]:
res = lp_solve(points_df, recipe_df, base_inputs_irl, sort_by_col = 'all', force_excess_unsinkable_usage = 'all')

## Constraints

,iron-ingot,alt-iron-ingot,alt-pure-iron-ingot,alt-leached-iron-ingot,copper-ingot,alt-copper-ingot,alt-pure-copper-ingot,alt-leached-copper-ingot,steel-ingot,alt-steel-ingot,alt-compacted-steel-ingot,alt-coke-steel-ingot,iron-plate,alt-steel-cast-plate,alt-iron-plate,iron-rod,alt-iron-rod,alt-aluminum-rod,wire,alt-iron-wire,alt-caterium-wire,alt-fused-wire,cable,alt-cable,alt-quickwire-cable,alt-coated-cable,concrete,alt-concrete,alt-rubber-concrete,alt-wet-concrete,screw,alt-screw,alt-steel-screw,biomass-from-alien-protein,solid-biofuel,steel-beam,alt-steel-beam,steel-pipe,alt-molded-steel-pipe,alt-iron-pipe,reinforced-iron-plate,alt-bolted-iron-plate,alt-stitched-iron-plate,alt-adhered-iron-plate,rotor,alt-rotor,alt-copper-rotor,modular-frame,alt-modular-frame,alt-modular-frame2,...,packaged-liquid-biofuel-trash,packaged-nitric-acid-trash,packaged-nitrogen-gas-trash,packaged-oil-trash,packaged-rocket-fuel-trash,packaged-sulfuric-acid-trash,packaged-turbofuel-trash,packaged-water-trash,petroleum-coke-trash,plastic-trash,plutonium-fuel-rod-trash,polymer-resin-trash,pressure-conversion-cube-trash,pulse-nobelisk-trash,quartz-crystal-trash,quickwire-trash,radio-control-unit-trash,raw-quartz-trash,reanimated-sam-trash,reinforced-iron-plate-trash,rifle-ammo-trash,rotor-trash,rubber-trash,sam-trash,sam-fluctuator-trash,screw-trash,shatter-rebar-trash,silica-trash,singularity-cell-trash,smart-plating-trash,smokeless-powder-trash,solid-biofuel-trash,stator-trash,steel-beam-trash,steel-ingot-trash,steel-pipe-trash,stun-rebar-trash,sulfur-trash,supercomputer-trash,superposition-oscillator-trash,thermal-propulsion-rocket-trash,time-crystal-trash,turbo-motor-trash,turbo-rifle-ammo-trash,uranium-trash,uranium-cell-trash,uranium-fuel-rod-trash,versatile-framework-trash,wire-trash,CONSTRAINT_VALUE
adaptive-control-unit,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ai-expansion-server,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ai-limiter,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
alclad-aluminum-sheet,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
alien-power-matrix,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

FOUND WASTE:  ['alien-protein', 'alumina-solution', 'crude-oil', 'dark-matter-residue', 'dissolved-silica', 'encased-plutonium-cell', 'excited-photonic-matter', 'ficsonium', 'ficsonium-fuel-rod', 'fuel', 'heavy-oil-residue', 'ionized-fuel', 'liquid-biofuel', 'nitric-acid', 'nitrogen-gas', 'non-fissile-uranium', 'plutonium-pellet', 'plutonium-waste', 'power-shard', 'rocket-fuel', 'somersloop', 'sulfuric-acid', 'turbofuel', 'uranium-waste', 'water']


## Output

#### qty_produce

...

,name,key_name,category,time,ingredients,products,power_range,power_rate,id,qty_produce,score_output,power_work
1035,ballistic-warp-drive Trash,ballistic-warp-drive-trash,sink,0.000000,"[['ballistic-warp-drive', 1]]",[],nan,0.000000,1035,115.000000,332963410.000000,0.000000
1033,assembly-director-system Trash,assembly-director-system-trash,sink,0.000000,"[['assembly-director-system', 1]]",[],nan,0.000000,1033,180.000000,90031680.000000,0.000000
1026,ai-expansion-server Trash,ai-expansion-server-trash,sink,0.000000,"[['ai-expansion-server', 1]]",[],nan,0.000000,1026,101.000000,60362852.000000,0.000000
1101,plutonium-fuel-rod Trash,plutonium-fuel-rod-trash,sink,0.000000,"[['plutonium-fuel-rod', 1]]",[],nan,0.000000,1101,5.000000,765920.000000,0.000000
1119,singularity-cell Trash,singularity-cell-trash,sink,0.000000,"[['singularity-cell', 1]]",[],nan,0.000000,1119,2.000000,229350.000000,0.000000
1025,adaptive-control-unit Trash,adaptive-control-unit-trash,sink,0.000000,"[['adaptive-control-unit', 1]]",[],nan,0.000000,1025,1.000000,76368.000000,0.000000
1098,packaged-water Trash,packaged-water-trash,sink,0.000000,"[['packaged-water', 1]]",[],nan,0.000000,1098,450.000000,58500.000000,0.000000
1057,em-control-rod Trash,em-control-rod-trash,sink,0.000000,"[['em-control-rod', 1]]",[],nan,0.000000,1057,7.000000,17920.000000,0.000000
1136,uranium-cell Trash,uranium-cell-trash,sink,0.000000,"[['uranium-cell', 1]]",[],nan,0.000000,1136,57.000000,8379.000000,0.000000
1097,packaged-turbofuel Trash,packaged-turbofuel-trash,sink,0.000000,"[['packaged-turbofuel', 1]]",[],nan,0.000000,1097,4.000000,2280.000000,0.000000


#### power_work

,name,key_name,category,time,ingredients,products,power_range,power_rate,id,qty_produce,score_output,power_work
257,Iron Ore,iron-ore,base,0.000833,[],"[['iron-ore', 1]]",nan,45.000000,257,92097.000000,0.000000,4144365.000000
267,Water,water,base,0.001667,[],"[['water', 1]]",nan,20.000000,267,195898.000000,0.000000,3917960.000000
258,Limestone,limestone,base,0.000833,[],"[['limestone', 1]]",nan,45.000000,258,69294.000000,0.000000,3118230.000000
262,Coal,coal,base,0.000833,[],"[['coal', 1]]",nan,45.000000,262,42300.000000,0.000000,1903500.000000
266,Nitrogen Gas,nitrogen-gas,base,0.001667,[],"[['nitrogen-gas', 1]]",nan,150.000000,266,12000.000000,0.000000,1800000.000000
259,Copper Ore,copper-ore,base,0.000833,[],"[['copper-ore', 1]]",nan,45.000000,259,36900.000000,0.000000,1660500.000000
261,Caterium Ore,caterium-ore,base,0.000833,[],"[['caterium-ore', 1]]",nan,45.000000,261,14999.000000,0.000000,674955.000000
218,Time Crystal,time-crystal,converting,10.000000,"[['diamonds', 2]]","[['time-crystal', 1]]",nan,250.000000,218,2592.000000,0.000000,648000.000000
263,Raw Quartz,raw-quartz,base,0.000833,[],"[['raw-quartz', 1]]",nan,45.000000,263,13494.000000,0.000000,607230.000000
260,Bauxite,bauxite,base,0.000833,[],"[['bauxite', 1]]",nan,45.000000,260,12296.000000,0.000000,553320.000000


#### slooped

,name,key_name,category,time,ingredients,products,power_range,power_rate,id,qty_produce,score_output,power_work
961,Pressure Conversion Cube,pressure-conversion-cube-slooped-overclocked,crafting2,60.000000,"[['fused-modular-frame', 2.5], ['radio-control-unit', 5.0], ['somersloop', 2]]","[['pressure-conversion-cube', 5.0]]",nan,201.464852,961,1.000000,0.000000,201.464852
974,Thermal Propulsion Rocket,thermal-propulsion-rocket-slooped-overclocked,crafting3,120.000000,"[['modular-engine', 12.5], ['turbo-motor', 5.0], ['cooling-system', 15.0], ['fused-modular-frame', 5.0], ['somersloop', 4]]","[['thermal-propulsion-rocket', 10.0]]",nan,738.704459,974,3.000000,0.000000,2216.113377
1005,Ballistic Warp Drive,ballistic-warp-drive-slooped-overclocked,crafting3,60.000000,"[['thermal-propulsion-rocket', 2.5], ['singularity-cell', 12.5], ['superposition-oscillator', 5.0], ['dark-matter-crystal', 100.0], ['somersloop', 4]]","[['ballistic-warp-drive', 5.0]]",nan,738.704459,1005,23.000000,0.000000,16990.202560


In [85]:
484520685 / 480345879

1.0086912497483922